# RQ2: In which phase of the technology life cycle path are the technologies?
- Interest in topic: #Papers on a topic over time 
- Parallel dazu: Interest in topic: Number of citations to papers on a topic over time
- Publikationstyp: Workshop -> Conference -> Journal 

## Bibliotheken einbinden

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Daten einlesen

In [2]:
data = pd.read_csv('./data/data.csv',usecols = ['Publication Year','Article Citation Count','Author Keywords','Publisher'], dtype={'Author Keywords': str, 'Publisher': str})
data.head()

,Publication Year,Author Keywords,Article Citation Count,Publisher
0,2016,software engineering education;athletic softwa...,5.0,IEEE
1,2009,software engineering;knowledge management;lear...,28.0,IEEE
2,2016,global software engineering;agile software dev...,11.0,IEEE
3,2011,software engineering;computer and information ...,11.0,IEEE
4,2017,automotive software engineering;ASE;AUTOSAR;st...,4.0,IEEE


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2165 entries, 0 to 2164
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Publication Year        2165 non-null   int64  
 1   Author Keywords         1976 non-null   object 
 2   Article Citation Count  1578 non-null   float64
 3   Publisher               2165 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 67.8+ KB


In [4]:
def IntrestInTopic(year):
    databyyear = data[data['Publication Year'] == year]
    

In [5]:
def FilterListErstellen(l):
    wordlist = []
    for x in l:
        wordlist.append(x[0])
    return pd.Series(wordlist,dtype=str)
    

In [6]:

citation_matrix_topics = data.pivot_table(index='Author Keywords', 
                        columns='Publication Year', 
                        values='Article Citation Count',
                        aggfunc=np.sum,
                        fill_value = 0)
    
citation_matrix_topics.head()
#citation_matrix_topics.info()

Publication Year,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Author Keywords,,,,,,,,,,,,,
- D.2.1Requirements/Specifications D.2.9.d Initiation and scope definition D.2.9 Management;Requirements/specifications;initiation and scope definition;management,0,0,0,0,0,0,0,12,0,0,0,0,0
11th International Symposium on Empirical Software Engineering and Measurement;13th International Conference on Predictive Models and Data Analytics in Software Engineering;21st International Systems and Software Product Line Conference;empirical software engineering;predictive models;data analytics;software product lines;product line engineering;developer productivity;issue resolution;agile development;microservices;software engineering;software development;Practitioners’ Digest,0,0,0,0,0,0,0,0,0,0,0,0,0
33rd International Conference on Software Maintenance and Evolution;ICSME 17;32nd International Conference on Automated Software Engineering;ASE 17;software maintenance;software evolution;automated software engineering;flaky tests;technical debt;self-admitted technical debt;SATD;QA bots;regular expressions;regexes;software engineering;software development;Practitioners’ Digest,0,0,0,0,0,0,0,0,0,0,0,0,0
40th International Conference on Software Engineering;17th International Conference on Software Reuse;gender;gender stereotypes;GenderMag;sentiment;sentiment analysis;Stack Overflow;Senti4SD;software product lines;SPL;software product line engineering;SPLE;variability management;safety-critical systems;software engineering education;software reuse;software development;software engineering;Practitioners’ Digest,0,0,0,0,0,0,0,0,0,1,0,0,0
API Documentation;Code Sample;Usage Scenario;Stack Overflow;Crowd knowledge,0,0,0,0,0,0,0,0,0,0,3,0,0


In [7]:
data["count"] = 1
matrix_topic_count = data.pivot_table(index='Author Keywords', 
                        columns='Publication Year', 
                        values='count',
                        aggfunc=np.sum,
                        fill_value = 0)

In [8]:
def KeywordTrennen(matrix,cnt):
    words = {}
    data = pd.DataFrame()
    i = 0

    for year in range(2009,2022,1):
        for keywords,count in matrix[year].items():
            i = i + 1
            keywordlist = keywords.split(';')
            if count > 0:
                for keyword in keywordlist:
                    #if keyword == 'software engineering':
                    #    print(f"{keywords}\n {count}\n {keyword}")
                    cnt[keyword] += 1
                    keys = list(words.keys())
                    if keyword not in keys:
                        words[keyword] = 0
                    words[keyword] += count

        seriesyear = pd.Series(words).to_frame()
        seriesyear.columns = [year]
        data = data.merge(seriesyear, how= 'outer' , left_index = True, right_index = True)
        words = {}   
    return data   

In [9]:
cnt_citation = Counter()

In [10]:
keyword_citation = KeywordTrennen(citation_matrix_topics,cnt_citation)  

In [11]:
keyword_citation 

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
automatic programming,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
coding tools and techniques,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
distribution,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
refactoring tools,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
workflow automation,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
workflow management,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
workflow-based systems,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
working time of developers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [12]:
anzahl_anzeigen = 50

In [13]:
cnt_citation.most_common(anzahl_anzeigen)

[('software engineering', 533),
 ('software development', 323),
 ('software architecture', 65),
 ('software', 50),
 ('software testing', 37),
 ('software quality', 34),
 ('requirements engineering', 31),
 ('requirements', 24),
 ('empirical study', 24),
 ('DevOps', 24),
 ('empirical software engineering', 23),
 ('machine learning', 23),
 ('refactoring', 23),
 ('software maintenance', 20),
 ('Software engineering', 19),
 ('agile software development', 19),
 ('agile', 19),
 ('software design', 18),
 ('reliability', 18),
 ('software product lines', 17),
 ('agile development', 17),
 ('security', 17),
 ('static analysis', 17),
 ('Internet of Things', 17),
 ('model checking', 16),
 ('Software testing', 16),
 ('cloud computing', 16),
 ('survey', 16),
 ('mining software repositories', 16),
 ('Android', 16),
 ('open source', 15),
 ('verification', 15),
 ('technical debt', 15),
 ('software requirements', 15),
 ('software architect', 14),
 ('mobile apps', 14),
 ('Model checking', 13),
 ('reverse e

In [14]:
filterlist = FilterListErstellen(list(cnt_citation.most_common(anzahl_anzeigen)))

In [15]:
keyword_citation_data = keyword_citation[keyword_citation.index.isin(filterlist)].fillna(0)

In [16]:
keyword_citation_data.head(anzahl_anzeigen)

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Android,0.0,0.0,0.0,0.0,0.0,125.0,136.0,62.0,28.0,0.0,8.0,2.0,8.0
DevOps,0.0,0.0,0.0,0.0,0.0,0.0,97.0,499.0,6.0,27.0,3.0,0.0,0.0
Internet of Things,0.0,0.0,0.0,0.0,0.0,0.0,27.0,198.0,332.0,9.0,0.0,1.0,0.0
Model checking,274.0,0.0,0.0,0.0,5.0,16.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0
Software engineering,31.0,4.0,109.0,112.0,106.0,0.0,58.0,0.0,16.0,0.0,6.0,0.0,2.0
Software testing,0.0,61.0,46.0,74.0,47.0,101.0,95.0,14.0,5.0,0.0,7.0,16.0,0.0
agile,0.0,22.0,88.0,33.0,2.0,10.0,20.0,41.0,6.0,20.0,0.0,1.0,0.0
agile development,0.0,47.0,0.0,57.0,0.0,0.0,0.0,12.0,13.0,20.0,0.0,0.0,0.0
agile software development,219.0,19.0,80.0,0.0,55.0,20.0,10.0,22.0,5.0,25.0,0.0,0.0,0.0
architecture,0.0,22.0,21.0,2.0,0.0,16.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
cnt_topic = Counter()

In [18]:
keyword_count = KeywordTrennen(matrix_topic_count,cnt_topic)  

In [19]:
filterlist = FilterListErstellen(list(cnt_topic.most_common(anzahl_anzeigen)))

In [20]:
keyword_count_data = keyword_count[keyword_count.index.isin(filterlist)].fillna(0)

In [21]:
keyword_count_data.head(anzahl_anzeigen)

,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Android,0.0,0.0,0.0,0.0,0.0,3.0,3.0,4.0,2.0,0.0,1.0,3.0,3.0
DevOps,0.0,0.0,0.0,0.0,0.0,0.0,3.0,8.0,4.0,12.0,1.0,2.0,0.0
IEEE Software,2.0,0.0,3.0,1.0,0.0,0.0,4.0,4.0,4.0,7.0,0.0,0.0,0.0
Internet of Things,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,8.0,3.0,0.0,1.0,0.0
Software Engineering,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,3.0,1.0
Software Engineering Radio,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,3.0,5.0,0.0,0.0,0.0
Software engineering,2.0,1.0,4.0,2.0,3.0,0.0,2.0,0.0,2.0,0.0,3.0,0.0,2.0
Software testing,0.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,0.0,2.0,6.0,2.0
UML,0.0,2.0,2.0,1.0,1.0,1.0,2.0,3.0,2.0,0.0,1.0,0.0,0.0
agile,0.0,2.0,4.0,1.0,2.0,2.0,1.0,4.0,1.0,3.0,0.0,1.0,0.0


In [22]:
cnt_topic.most_common(anzahl_anzeigen)

[('software engineering', 698),
 ('software development', 430),
 ('software architecture', 81),
 ('software', 65),
 ('software testing', 49),
 ('requirements engineering', 41),
 ('software quality', 40),
 ('DevOps', 30),
 ('empirical software engineering', 28),
 ('empirical study', 28),
 ('software maintenance', 27),
 ('machine learning', 27),
 ('refactoring', 27),
 ('requirements', 26),
 ('agile development', 26),
 ('IEEE Software', 24),
 ('software design', 24),
 ('software requirements', 22),
 ('agile software development', 22),
 ('technical debt', 22),
 ('static analysis', 22),
 ('Software engineering', 21),
 ('Software testing', 21),
 ('Software Engineering', 20),
 ('software product lines', 20),
 ('reliability', 20),
 ('agile', 20),
 ('mining software repositories', 20),
 ('survey', 19),
 ('cloud computing', 19),
 ('Android', 19),
 ('model checking', 18),
 ('security', 18),
 ('Internet of Things', 18),
 ('architecture', 17),
 ('software analytics', 17),
 ('software reliability', 

In [23]:
#- Publikationstyp: Workshop -> Conference -> Journal 
#data['Author Keywords'] = data['Author Keywords'].str.split(';',expand=False)

#data['Author Keywords'].head()

#data[data['Author Keywords'].str.split(';',expand=False).Contains]
#Mit isin Liste abfragen
#Erst Topic Filtern und dann weiter Arbeiten

#test = {}

#words = Counter()
#data[data['Publication Year'] == year]
#for year in range(2009,2021,1):
#    for columns in data[data['Publication Year'] == year].itertuples():

#        text = columns[2] #
#        if type(text) is str:
#            for topic in text.split(';'):
#                words[topic] += 1

#        test[year] = list(words)

    #print(words.most_common(40))   

In [24]:
i= 1
for x,y in citation_matrix_topics[2009].items():
    print(f"{x}\n {y}\n {i}")
    i = i +1

- D.2.1Requirements/Specifications D.2.9.d Initiation and scope definition D.2.9 Management;Requirements/specifications;initiation and scope definition;management
 0
 1
11th International Symposium on Empirical Software Engineering and Measurement;13th International Conference on Predictive Models and Data Analytics in Software Engineering;21st International Systems and Software Product Line Conference;empirical software engineering;predictive models;data analytics;software product lines;product line engineering;developer productivity;issue resolution;agile development;microservices;software engineering;software development;Practitioners’ Digest
 0
 2
33rd International Conference on Software Maintenance and Evolution;ICSME 17;32nd International Conference on Automated Software Engineering;ASE 17;software maintenance;software evolution;automated software engineering;flaky tests;technical debt;self-admitted technical debt;SATD;QA bots;regular expressions;regexes;software engineering;sof

In [25]:
#import spacy
#from spacy.symbols import NOUN


#nlp = spacy.load("en_core_web_sm")

#cnt = Counter()
#tokenlist = []

#for columns in data.itertuples():

#    text = columns[2] #1 = title 2 = year 3 = abstract
    #print(type(abstract))
    # Verarbeite den Text
#    if type(text) is str:
#        doc = nlp(text)
        
#        for token in doc:
            # Greife auf den Text, die Wortart und die Dependenzrelation des Tokens zu
#            token_text = token.text
#            token_pos = token.pos_
#            token_dep = token.dep_
#            token_ent = token.ent_type_
            
#            if token.pos == NOUN:
#                if not token.is_stop:
#                    #print(f"{token_text} | {token_ent} | {token.tag_}")
#                    cnt[token_text] += 1
#                    tokenlist.append([token_text,columns[2],token_ent])
